In [2]:
!pip install boto3


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!aws transcribe start-transcription-job \

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: --transcription-job-name, --media


In [8]:
!aws s3 ls s3://qtm350finalbucket/audio\ files/

2023-04-16 20:27:39     122154 1.m4a
2023-04-16 20:27:37      67355 10.m4a
2023-04-16 20:27:37      79172 11.m4a
2023-04-16 20:27:38      98437 12.m4a
2023-04-16 20:27:38      93029 13.m4a
2023-04-16 20:27:39     115640 14.m4a
2023-04-16 20:27:39      78655 15.m4a
2023-04-16 20:27:39      96745 2.m4a
2023-04-16 20:27:40     103714 3.m4a
2023-04-16 20:27:41     105909 4.m4a
2023-04-16 20:27:41      85590 5.m4a
2023-04-16 20:27:40      52323 6.m4a
2023-04-16 20:27:40      78459 7.m4a
2023-04-16 20:27:36      45301 8.m4a
2023-04-16 20:27:37      60720 9.m4a


In [13]:
!pip install pydub
!apt-get update
!apt-get install -y ffmpeg



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Get:1 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [478 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7909 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]
Fetched 8609 kB in 2s (5469 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  i965-va-driver intel-media-va-driver libaacs0 libaom0 libasound2
  libasound2-data libass9 libasyncns0 libatomic1 libavc1394-0 libavcodec58
  libavdevice58 libavfilter7 libavformat58 libavresample4 libavutil56
  lib

In [14]:
import boto3
import os
from pydub import AudioSegment

# Create a boto3 S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Create a boto3 Transcribe client
transcribe = boto3.client('transcribe', region_name='us-east-1')

# List all objects in the "audio files" folder
bucket_name = 'qtm350finalbucket'
folder_name = 'audio files/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

audio_files = [item['Key'] for item in response['Contents']]

# Download, convert, and upload each audio file
for file_name in audio_files:
    file_uri = f"s3://{bucket_name}/{file_name}"
    local_m4a_file = os.path.basename(file_name)
    local_mp3_file = local_m4a_file.replace('.m4a', '.mp3')
    mp3_file_name = file_name.replace('.m4a', '.mp3')
    mp3_file_uri = f"s3://{bucket_name}/{mp3_file_name}"

    # Download the m4a file
    s3.download_file(bucket_name, file_name, local_m4a_file)

    # Convert m4a to mp3
    audio = AudioSegment.from_file(local_m4a_file, format="m4a")
    audio.export(local_mp3_file, format="mp3")

    # Upload the mp3 file
    s3.upload_file(local_mp3_file, bucket_name, mp3_file_name)

    # Start transcription job
    job_name = f"transcribe-job-{os.path.basename(mp3_file_name)}"

    transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        LanguageCode='en-US',
        MediaFormat='mp3',
        Media={'MediaFileUri': mp3_file_uri},
        OutputBucketName=bucket_name
    )
    
    print(f"Started transcription job for {mp3_file_name}")

    # Remove local files
    os.remove(local_m4a_file)
    os.remove(local_mp3_file)


Started transcription job for audio files/1.mp3
Started transcription job for audio files/10.mp3
Started transcription job for audio files/11.mp3
Started transcription job for audio files/12.mp3
Started transcription job for audio files/13.mp3
Started transcription job for audio files/14.mp3
Started transcription job for audio files/15.mp3
Started transcription job for audio files/2.mp3
Started transcription job for audio files/3.mp3
Started transcription job for audio files/4.mp3
Started transcription job for audio files/5.mp3
Started transcription job for audio files/6.mp3
Started transcription job for audio files/7.mp3
Started transcription job for audio files/8.mp3
Started transcription job for audio files/9.mp3
